# -----------------------------------------------------------------------------------------
# 1. Setup
# -----------------------------------------------------------------------------------------

#### Import Packages

In [2]:
# For reading in Json file
import json

# general packages
import pandas as pd
import numpy as np
import time

# For selecting period of valuation table
from dateutil.relativedelta import relativedelta

#### Import Functions

In [3]:
# Load the autoreload extension
%load_ext autoreload
%autoreload 1

# Import functions for scraping the data
%aimport get_data
import get_data as gd

# Import functions for analysing the data
%aimport analyse_portfolio
import analyse_portfolio as ap

#### Configure Notebook

In [4]:
# Update the default number of rows that are displayed
pd.set_option('display.max_rows', 500)

# -----------------------------------------------------------------------------------------
# 2. Read in data
# -----------------------------------------------------------------------------------------

#### Run Functions which Scrape Data to Populate Files

In [ ]:
gd.create_company_ticker_to_name_map()
gd.create_exchange_ticker_to_name_map()

#### Read in Manually Populated Transactions Table

In [12]:
raw_transactions = gd.read_in_raw_transactions_data()

#### Use Scraped Data to Add Columns to Transactions Data

In [13]:
# Add company name
transactions_comp_name = gd.create_company_name_col_using_ticker(raw_transactions)

The following stocks did not have an associated company name:
|     | stock_ticker   | exchange_ticker   |
|----:|:---------------|:------------------|
|  45 | SVE            | TSV               |
|  49 | NOVR           | TSV               |
|  52 | MMG            | TSV               |
|  53 | RSLV           | TSV               |
|  59 | LGD            | TOR               |
|  61 | SSV            | TSV               |
| 101 | ELEC           | TSV               |


In [14]:
# Add Exchange name
transactions_exch_name = gd.create_exchange_name_nd_curency_cols_using_ticker(transactions_comp_name)

In [15]:
# Add aggregate price columns
transactions = gd.add_aggregate_price_columns(transactions_exch_name)

# -----------------------------------------------------------------------------------------
# 3. Create Stock Valuation Table
# -----------------------------------------------------------------------------------------

In [6]:
unique_tickers_list = list(transactions["stock_ticker"].drop_duplicates())
ticker_start_date_list = [str(min(transactions[transactions["stock_ticker"] == ticker]["date"]))[:10] for ticker in unique_tickers_list]
gd.create_table_of_historic_stock_prices(unique_tickers_list, ticker_start_date_list)

- LGD: No data found for this date range, symbol may be delisted
LGD not found
- MMG: No data found for this date range, symbol may be delisted
MMG not found
- NOVR: No data found, symbol may be delisted
NOVR not found
- RSLV: No data found, symbol may be delisted
RSLV not found
- SSV: No data found for this date range, symbol may be delisted
SSV not found
- SVE: No data found for this date range, symbol may be delisted
SVE not found


In [7]:
unique_currency_list = list(transactions["currency"].drop_duplicates())
start_date = min(transactions["date"]).date()
base_currency = 'EUR'
gd.create_exchange_rate_to_date_map(base_currency, unique_currency_list, start_date)

In [8]:
stock_data = ap.get_specific_stock(transactions, "GOLD", "NSY")

In [9]:
stock_data.dtypes

date                        datetime64[ns]
stock_ticker                        object
exchange_ticker                     object
num_shares                           int64
share_price                        float64
exchange_rate                      float64
company_name                        object
currency                            object
exchange_name                       object
share_cost_in_$                    float64
currency_exchange_fee              float64
fixed_transaction_fee              float64
variable_transaction_fee           float64
total_outgoing_in_eur              float64
share_cost_in_eur                  float64
total_shares_to_date                 int64
break_even_price                   float64
dtype: object

In [35]:
daily_account_balance_df = ap.calculate_daily_account_balance(stock_data)

In [50]:
# {"day": relativedelta(days=1),
#  "week": relativedelta(weeks=1),
#  "month": relativedelta(months=1),
#  "quarter": relativedelta(months=3),
#  "year": relativedelta(years=1),
# }

base_currency = "EUR"
ap.create_valuation_table(stock_data, relativedelta(weeks=1), base_currency)

TypeError: unsupported operand type(s) for +: 'int' and 'str'